# 4.3.2 有価証券報告書内のMD＆Aが記載されているhtmファイルを抽出　

　Zipファイルで送られてきた大量の有価証券書のデータをPythonにより自動的に解凍した。その後、有価証券報告書内のMD＆A情報のみが記載されたhtmファイルのみを抽出をした。

In [15]:
import pandas as pd
import glob
import os
import zipfile
from datetime import datetime as dt
import bs4 
import re

In [16]:
def make_industry_zip_list_hash(data_frame, data_frame_name):
    
    industry_zip_list_hash = {}
    industry_li = make_type_of_industry_list(data_frame)
    zip_files = call_zip_files(data_frame_name)
    for industry in industry_li :
        industry_zip_list_hash[industry]= list(filter(lambda x: industry in x , zip_files))
    
    return   industry_zip_list_hash
    

def make_type_of_industry_list(data_frame : pd.DataFrame, industry_col="[業種（東証）]"):
    return ["サービス業"]


def call_zip_files(data_frame_name):
    zip_files = glob.glob(f"../**/SampleData/{data_frame_name}/**/**.zip", recursive=True)
    return zip_files

def call_unziped_htm_files_dir(data_frame_name: str):

    unziped_htm_files_dir = os.getcwd()+  "/UnzipedHtmFiles"
    if not os.path.exists(unziped_htm_files_dir) :
        os.mkdir(unziped_htm_files_dir)

    unziped_htm_files_dir_with_df_name = unziped_htm_files_dir + f"/{data_frame_name}"
    if not os.path.exists(unziped_htm_files_dir_with_df_name) :
        os.mkdir(unziped_htm_files_dir_with_df_name)
    
    return unziped_htm_files_dir_with_df_name


#--------------------------------------------------------------------
def unzip_html_to_unziped_htm_files_dir(industry_zip_list_hash, filepath_unziped):
    
    sum_files_len = sum(map(len, industry_zip_list_hash.values()))
    zip_files_len = 1
    
    for industy_name, zip_files in industry_zip_list_hash.items() :
        zip_files_len += len(zip_files)
        
        industry_dir = call_industry_dir(filepath_unziped, industy_name)
    
        for idx ,zip_file in enumerate(zip_files):
            try:
                with zipfile.ZipFile(zip_file) as existing_zip:

                    candidate_files = existing_zip.namelist()

                    for c_f in candidate_files:
                        basename = os.path.basename(c_f)
                        #第２部のもの(MD&Aが記載されている見出しの部分)を判定
                        if re.match(r'01020.*htm', basename) != None :
                            print(c_f)
                            date_str = c_f[-20:-10]
                            date_dt = dt.strptime(date_str, '%Y-%m-%d')

                            existing_zip.extract(c_f, industry_dir)

                    print(f"{idx + 1} / {len(zip_files)} ||  {zip_files_len - 1}/{sum_files_len}")
            except Exception :
                print(zip_file)
                with open("unzipping_html_error.txt", "a") as f:
                    f.write(zip_file + "\n")
                
                
def call_industry_dir(filepath_unziped, industry_name: str):
    industry_dir =  filepath_unziped+  f"/{industry_name}"
    if not os.path.exists(industry_dir) :
        os.mkdir(industry_dir)
    return industry_dir

In [17]:

# data_frame_name = "ave_top_20"
data_frame_name = "ave_worst_20"
# data_frame_name = "2021_top_20"
# data_frame_name = "2021_worst_20"

data_frame = pd.read_csv("/home/jovyan/work/1CallingEdinetApi"+f"/EdinetIdxFiles/{data_frame_name}.csv", skiprows=0)

In [19]:
industry_zip_list_hash = make_industry_zip_list_hash(data_frame, data_frame_name)
filepath_unziped = call_unziped_htm_files_dir(data_frame_name)
unzip_html_to_unziped_htm_files_dir(industry_zip_list_hash, filepath_unziped)

XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E04707-000_2018-03-31_01_2018-06-28_ixbrl.htm
1 / 111 ||  111/111
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E03379-000_2018-03-31_01_2018-06-18_ixbrl.htm
2 / 111 ||  111/111
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E04704-000_2018-03-31_01_2018-06-29_ixbrl.htm
3 / 111 ||  111/111
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E24017-000_2018-03-31_01_2018-06-20_ixbrl.htm
4 / 111 ||  111/111
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E04348-000_2018-03-31_01_2018-06-20_ixbrl.htm
5 / 111 ||  111/111
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E04990-000_2018-03-31_01_2018-06-20_ixbrl.htm
6 / 111 ||  111/111
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E31103-000_2018-03-31_01_2018-06-21_ixbrl.htm
7 / 111 ||  111/111
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E05456-000_2018-03-31_01_2018-06-21_ixbrl.htm
8 / 111 ||  111/111
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E05175-000_2018-03-31_

71 / 111 ||  111/111
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E05145-000_2019-03-31_01_2019-06-28_ixbrl.htm
72 / 111 ||  111/111
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E02057-000_2019-03-31_01_2019-06-28_ixbrl.htm
73 / 111 ||  111/111
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E04704-000_2019-03-31_01_2019-07-01_ixbrl.htm
74 / 111 ||  111/111
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E04348-000_2020-03-31_01_2020-06-17_ixbrl.htm
75 / 111 ||  111/111
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E04990-000_2020-03-31_01_2020-06-17_ixbrl.htm
76 / 111 ||  111/111
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E05456-000_2020-03-31_01_2020-06-18_ixbrl.htm
77 / 111 ||  111/111
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E04899-000_2020-03-31_01_2020-06-18_ixbrl.htm
78 / 111 ||  111/111
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E01987-000_2020-03-31_01_2020-06-26_ixbrl.htm
79 / 111 ||  111/111
XBRL/PublicDoc/0102010_honbun_jpcrp030000-a